In [1]:
#!/usr/bin/env python
# coding: utf-8
import math
import torch
import torch.nn.functional as F
import torch.optim as optim
import operator, functools
from sacremoses import MosesPunctNormalizer, MosesTokenizer
from nltk.tokenize import word_tokenize
from tqdm import tqdm
# from accelerate import Accelerator
import nltk
import sys
import pandas as pd
import wandb
from random import randrange
import random
import json
import os
import random
import tqdm
import sys
import time
import glob
import numpy as np
import torch
# import utils
import logging
import gc
import argparse

import torch
import torch.nn as nn
import torch.utils
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch import optim
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler

from tokenizers import Tokenizer
import datasets
from sacremoses import MosesPunctNormalizer, MosesTokenizer
# from data_set import *
from transformers import GPT2Tokenizer, AutoTokenizer, AutoModelForSeq2SeqLM

import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab,vocab
from torchtext.utils import download_from_url, extract_archive
import io


from models.GPT import *
from models.mBART import *
from models.RNN_sp import *
from models.attention_params import *

from models.hyperparams import *
from models.Transformer import *

from betty.engine import Engine
from betty.problems import ImplicitProblem
from betty.configs import Config, EngineConfig

import sentencepiece as spm
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
sp_src = spm.SentencePieceProcessor(model_file='upload_datasets/flores/flores200_sacrebleu_tokenizer_spm.model')

In [5]:
sp_src.vocab_size()

256000

In [2]:
# parser = argparse.ArgumentParser("Machine Translation")
# parser.add_argument('--dataset_name', type=str, default='wmt14_gigafren', help='dataset_name')
# parser.add_argument('--num_beams', type=int, default=2, help='Beam Size')
# parser.add_argument('--max_source_length', type=int, default=50, help='Max Source length')
# parser.add_argument('--max_target_length', type=int, default=50, help='Max Target length')
# parser.add_argument('--pad_to_max_length', type=bool, default=False, help='Pad to max length')
# parser.add_argument('--ignore_pad_token_for_loss', type=bool, default=True, help='Ignore pad token for loss')
# parser.add_argument('--per_device_train_batch_size', type=int, default=64, help='Train Batch Size')
# parser.add_argument('--per_device_eval_batch_size', type=int, default=64, help='Eval Batch Size')

# parser.add_argument('--mbart_learning_rate', type=float, default=1e-3, help='mBART learning rate')
# parser.add_argument('--mbart_learning_rate_min', type=float, default=5e-4, help='mBART min learning rate')
# parser.add_argument('--mbart_momentum', type=float, default=0.9, help='mBART momentum')
# parser.add_argument('--mbart_weight_decay', type=float, default=0, help='mBART weigth decay')

# parser.add_argument('--gpt_learning_rate', type=float, default=1e-3, help='GPT learning rate')
# parser.add_argument('--gpt_learning_rate_min', type=float, default=5e-4, help='GPT min learning rate')
# parser.add_argument('--gpt_momentum', type=float, default=0.9, help='GPT momentum')
# parser.add_argument('--gpt_weight_decay', type=float, default=0, help='GPT weigth decay')

# parser.add_argument('--rnn_learning_rate_adam', type=float, default=1e-3, help='RNN learning rate')
# parser.add_argument('--rnn_learning_rate_min_adam', type=float, default=5e-4, help='RNN min learning rate')
# parser.add_argument('--rnn_momentum', type=float, default=0.0, help='RNN momentum')
# parser.add_argument('--rnn_weight_decay', type=float, default=5e-7, help='RNN weigth decay')
# parser.add_argument('--beta1', type=float, default=0.9, help='Beta 1')
# parser.add_argument('--beta2', type=float, default=0.999, help='Beta 2')
# parser.add_argument('--eps', type=float, default=1e-8, help='Epsilon')
# parser.add_argument('--rnn_learning_rate_sgd', type=float, default=1, help='RNN learning rate sgd')
# parser.add_argument('--rnn_learning_rate_min_sgd', type=float, default=0.1, help='RNN min learning rate sgd')
# parser.add_argument('--rnn_optimizer', type=str, default='adam', help='RNN Optimizer')

# parser.add_argument('--begin_epoch', type=int, default=0, help='Begin Epoch')
# parser.add_argument('--stop_epoch', type=int, default=5, help='Stop Epoch')
# parser.add_argument('--report_freq', type=int, default=50, help='Report Frequency')
# parser.add_argument('--gpu', type=int, default=0, help='GPU')
# parser.add_argument('--num_train_epochs', type=int, default=15, help='Train Epochs')
# parser.add_argument('--final_training', type=int, default=15, help='Train Epochs Final')

# parser.add_argument('--seed', type=int, default=seed_, help='Seed')

# parser.add_argument('--grad_clip', type=int, default=0.25, help='Grad clip')
# parser.add_argument('--A_momentum', type=float, default=0.9, help='A momentum')
# parser.add_argument('--A_learning_rate', type=float, default=1e-3, help='A learning rate')
# parser.add_argument('--A_weight_decay', type=float, default=1e-3, help='A weight decay')

# parser.add_argument('--B_momentum', type=float, default=0.9, help='B momentum')
# parser.add_argument('--B_learning_rate', type=float, default=1e-3, help='B learning rate')
# parser.add_argument('--B_weight_decay', type=float, default=1e-3, help='B weight decay')

# parser.add_argument('--lambda_par', type=float, default=0.5, help='Lambda parameter')
# parser.add_argument('--train_num_points', type=int, default=5000, help='Train number points')
# parser.add_argument('--val_num_points', type=int, default=None, help='Validation number points')
# parser.add_argument('--save', type=str, default='EXP_aug', help='Save dir')
# parser.add_argument('--start_from_checkpoint', action='store_true', help='Use checkpoint')
# parser.add_argument('--logging', action='store_true', help='Use logging')
# parser.add_argument('--weight_decay', type=float, default=0.0, help='Weight decay')
# parser.add_argument('--save_models', action='store_true', help='Save models')
# parser.add_argument('--update_arch', action='store_true', help='Save models')
# parser.add_argument('--update_gpt_bart', action='store_true', help='Save models')
# # parser.add_argument('--save_alpha', help='Save a and b parameters', action='store_true')
# parser.add_argument('--save_alpha', action='store_true',help='Save Alpha')
# parser.add_argument('--save_freq', type=int, default=1, help='Frequency of saving A and B')
# parser.add_argument('--log_file_name', type=str, default='log.txt', help='Log file name')
# parser.add_argument('--validation_freq', type=int, default=5, help='Validation Frequency')
# parser.add_argument('--full', type=str, default='', help='Full prefix')
# parser.add_argument('--fp16', action='store_true',help='Low Precision')
# parser.add_argument('--gradient_accumulation_steps', type = int, default = 1,help='gradient accumulation steps')
# parser.add_argument('--max_train_steps', type = int, default = None,help='Max train steps')
# parser.add_argument('--max_train_steps_free', type = int, default = None,help='Final Max train steps')
# parser.add_argument('--use_engine', action='store_true', help='Use betty engine')
# parser.add_argument('--sequential_train', action='store_true', help='Train Coplete model sequentially')
# parser.add_argument('--transformer_model', action='store_true', help='To use Transformer model')
# parser.add_argument('--no_aug', action='store_true', help='No augmentation')
# parser.add_argument('--translation_language', type=str, default='fr', help='Translation language')
# parser.add_argument('--baseline', type=str, default=None, help='Translation language')
# parser.add_argument('--exp_name', type=str, default=None, help='Experiment Name')
# parser.add_argument('--baseline_par', type=float, default=None, help='Baseline parameters')
# parser.add_argument('--gpt_optimizer', type=str, default=None, help='GPT Optimizer Name')
# parser.add_argument('--mbart_optimizer', type=str, default=None, help='Mbart Optimizer Name')
# parser.add_argument('--mbart_scheduler', type=str, default='cosine', help='Mbart scheduler Name')
# parser.add_argument('--gpt_scheduler', type=str, default='cosine', help='gpt scheduler Name')
# parser.add_argument('--summary_length', type=int, default=128, help='SOurce length')
# parser.add_argument('--article_length', type=int, default=128, help='SOurce length')
# parser.add_argument('--log_steps', type = int, default = 100,help='MLO log steps')
# parser.add_argument('--dataset_type', type=str, default='iwslt', help='Dataset Name')
# parser.add_argument('--A_optimizer', type=str, default=None, help='A, B Optimizer Name')
# parser.add_argument('--src_lang', type=str, default='cor', help='lang name')
# parser.add_argument('--trg_lang', type=str, default='eng', help='lang name')
# parser.add_argument('--batch_size', type=int, default=32, help='Batch Size')
# args = parser.parse_args()

In [3]:
class Args():
    def __init__(self):
        self.translation_language = 'de'
        self.train_num_points = -1
        self.train_num_points_back = 100000
        self.val_num_points = -1
        self.transformer_model = False
        self.weight_decay = 0
        self.epochs = 30
        self.epochs_back = 5
        self.optimizer_back = 'adam'
        self.lr_back = 2e-5
        self.gradient_accumulation_steps = 1
        self.back_model = 'opus'
        self.no_augmentation = False
        self.only_aug = False
        self.train_seq = True
        self.preprocessing_num_workers = 1
        self.epochs_on_mono = 5
    

In [4]:
args = Args()

In [5]:
args.src_lang = 'chm'
args.trg_lang = 'rus'

In [4]:
from datasets import load_dataset

dataset = load_dataset("flores", "sien")

FileNotFoundError: Couldn't find file at https://github.com/facebookresearch/flores/raw/main/floresv1/data/wikipedia_en_ne_si_test_sets.tgz

In [6]:
def read_data(split='train'):
    with open('upload_datasets/'+args.src_lang+'_'+args.trg_lang+'/'+split+'.src', 'r') as file:
        # Read the file line by line
        src_lines = file.readlines()
    with open('upload_datasets/'+args.src_lang+'_'+args.trg_lang+'/'+split+'.trg', 'r') as file:
        # Read the file line by line
        trg_lines = file.readlines()
        
    for i in range(len(src_lines)):
        src_lines[i] = src_lines[i].strip()
        trg_lines[i] = trg_lines[i].strip()
    return src_lines,trg_lines

train_data = read_data('train')
dev_data = read_data('dev')
test_data = read_data('test')


In [7]:
def load_tokenizer(src_lang=args.src_lang,trg_lang=args.trg_lang,n_src= '4k',n_trg='4k'):
    sp_src = spm.SentencePieceProcessor(model_file='upload_datasets/'+args.src_lang+'_'+args.trg_lang+'/'+'opusTC.'+args.src_lang+'.'+n_src+'.spm')
    sp_trg = spm.SentencePieceProcessor(model_file='upload_datasets/'+args.src_lang+'_'+args.trg_lang+'/'+'opusTC.'+args.trg_lang+'.'+n_trg+'.spm')
    sp_src.SetEncodeExtraOptions("bos:eos")
    sp_trg.SetEncodeExtraOptions("bos:eos")
    return sp_src,sp_trg

sp_src, sp_trg = load_tokenizer()

In [8]:
sp_trg

<sentencepiece.SentencePieceProcessor; proxy of <Swig Object of type 'sentencepiece::SentencePieceProcessor *' at 0x7f6358199570> >

In [9]:
def seed_torch(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch(seed_)

In [10]:
def data_process_train(data):
    data_process = []
    for  raw_src,raw_trg in zip(data[0],data[1]):
        rnn_src = sp_src.encode(raw_src)
        rnn_trg = sp_trg.encode(raw_trg)
        data_process.append((rnn_src,rnn_trg))
    return data_process

train_tokens = data_process_train(train_data)
dev_tokens = data_process_train(dev_data)
test_tokens = data_process_train(test_data)

In [11]:
args.batch_size = 32

def generate_batch(data_batch):
    rnn_src_ids, rnn_trg_ids = [],[]
    for (src_id,trg_id) in data_batch:
        rnn_src_ids.append(torch.tensor(src_id,dtype=torch.int64))
        rnn_trg_ids.append(torch.tensor(trg_id,dtype=torch.int64))
    rnn_src_ids = pad_sequence(rnn_src_ids, batch_first=True, padding_value=0)
    rnn_trg_ids = pad_sequence(rnn_trg_ids, batch_first=True, padding_value=0)    
    return rnn_src_ids,rnn_trg_ids


batch_size = args.batch_size

rnn_dataloader_train = DataLoader(train_tokens,batch_size=batch_size,
                        shuffle=False, collate_fn=generate_batch)
rnn_dataloader_val = DataLoader(dev_tokens,batch_size=batch_size,
                        shuffle=False, collate_fn=generate_batch)
rnn_dataloader_test = DataLoader(test_tokens,batch_size=batch_size,
                        shuffle=False, collate_fn=generate_batch)

In [12]:
len(sp_src)

3847

In [13]:
len(sp_trg)

4000

In [14]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

def create_rnn_model(src_tok,trg_tok):
    PAD_IDX = 0
    rnn_criterion=torch.nn.CrossEntropyLoss(ignore_index =PAD_IDX,reduction='mean')# reduction='mean'
    rnn_criterion=rnn_criterion.cuda()
#     if args.transformer_model:
#         rnn_model = TRANSFORMER_MODEL(rnn_criterion,rnn_tokenizer) 
#     else:
    rnn_model = RNN_MODEL(rnn_criterion,src_tok,trg_tok) 

    rnn_model.apply(init_weights)
    rnn_model = rnn_model.cuda()

    return rnn_model

def group_parameters(model):
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    return optimizer_grouped_parameters
device = "cuda" if torch.cuda.is_available() else "cpu"
from tqdm import tqdm

In [15]:
import evaluate

In [16]:
def train_target_to_source(model,data_loader,optimizer,rnn_scheduler,epochs):
    for t in tqdm(range(epochs)):
        model.train()
        train_loss = 0.0
        for j,batch in enumerate(tqdm(data_loader)):
            optimizer.zero_grad()
            mbart_src_ids,mbart_trg_ids = batch
            mbart_src_ids,mbart_trg_ids = mbart_src_ids.to(device),mbart_trg_ids.to(device)
            
            loss = model.loss(mbart_trg_ids,mbart_src_ids)
            train_loss+=loss.item()
            loss.backward()
            optimizer.step()
            rnn_scheduler.step()
        d = {'loss':train_loss/j}
#         wand_log.log(d,tag = 'target_to_source')

beam_width=1
temperature = 1.0
do_sample=False
repetition_penalty = 1.5
top_p = 0.9
top_k = 100
val_steps = 2
chrf_metric_1 = evaluate.load("chrf")
bleu_metric = evaluate.load("bleu")
meteor_metric = evaluate.load('meteor')
chrf_metric_2 = evaluate.load("chrf") 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [21]:
args.create_bt_data = True
args.create_path  = 'wiki.aa.rus-eng.rus'

In [22]:
from transformers import get_linear_schedule_with_warmup

In [25]:
args.max_gen_data = 200000

In [26]:
  
if args.create_bt_data:
    rnn_model_target_to_source = create_rnn_model(sp_trg,sp_src).to(device)
    mbart_params = rnn_model_target_to_source.parameters()
    # mbart_params = group_parameters(rnn_model_target_to_source)
    # if args.optimizer_back == 'adam':
    rnn_optimizer_target_to_source = optim.AdamW(mbart_params)

    num_update_steps_per_epoch = math.ceil(len(rnn_dataloader_train))
    max_train_steps = num_update_steps_per_epoch*args.epochs
    warmup_steps = 0.1*max_train_steps
    rnn_scheduler_target_to_source = get_linear_schedule_with_warmup(
                    rnn_optimizer_target_to_source, num_warmup_steps=warmup_steps, num_training_steps=max_train_steps
                )
    train_target_to_source(rnn_model_target_to_source,rnn_dataloader_train,rnn_optimizer_target_to_source,rnn_scheduler_target_to_source,20)#model,data_loader,optimizer,rnn_scheduler,epochs
    with open('upload_datasets/'+args.src_lang+'_'+args.trg_lang+'/'+args.create_path, 'r') as file:
        # Read the file line by line
        trg_lines = file.readlines()
            
    for i in range(len(trg_lines)):
        trg_lines[i] = trg_lines[i].strip()
    if len(trg_lines)>args.max_gen_data:
        trg_lines = trg_lines[:args.max_gen_data]
    def data_process_gen(data):    
        data_process = []
        for  raw_trg in trg_lines:
            rnn_trg = sp_trg.encode(raw_trg)
            data_process.append((rnn_trg))
        return data_process
       
    gen_tokens = data_process_gen(trg_lines) 
    def generate_batch_gen(data_batch):
        rnn_trg_ids = []
        for trg_id in data_batch:
            rnn_trg_ids.append(torch.tensor(trg_id,dtype=torch.int64))
        rnn_trg_ids = pad_sequence(rnn_trg_ids, batch_first=True, padding_value=0)    
        return rnn_trg_ids
    
    rnn_dataloader_gen = DataLoader(gen_tokens,batch_size=batch_size,
                        shuffle=False, collate_fn=generate_batch_gen)
    source_sents_l = []
    target_sents_l = []
    for i, batch in enumerate(rnn_dataloader_gen):
        trans_sentences = []
        labels_sentences = []
        src = batch.to(device)
#         print(src.shape)
        pred_sents = rnn_model_target_to_source.generate(src,beam_width=beam_width,temperature=temperature , do_sample=do_sample, repetition_penalty=repetition_penalty ,top_p=top_p, top_k=top_k)
        pred_sents = sp_src.decode(pred_sents.detach().cpu().numpy().tolist()) #deu
        source_sents = sp_trg.decode(src.detach().cpu().numpy().tolist()) #ido

        pred_sents = [ti.replace(" ⁇ ", "") for ti  in pred_sents] #deu
        source_sents = [ti.replace(" ⁇ ", "") for ti  in source_sents] #ido
        
        source_sents_l.extend(source_sents)
        target_sents_l.extend(pred_sents)
    print(source_sents_l[:5])
    print(target_sents_l[:5])
    # Open a file for writing
    with open('upload_datasets/'+args.src_lang+'_'+args.trg_lang+'/wiki.aa.'+args.src_lang+'-'+args.trg_lang+'.'+args.src_lang, 'w') as f:
        # Write each element of the list to the file
        for item in target_sents_l:
            f.write("%s\n" % item)
    with open('upload_datasets/'+args.src_lang+'_'+args.trg_lang+'/wiki.aa.'+args.src_lang+'-'+args.trg_lang+'.'+args.trg_lang, 'w') as f:
        # Write each element of the list to the file
        for item in source_sents_l:
            f.write("%s\n" % item)
            

        
rnn_model_source_to_target = create_rnn_model().to(device)
mbart_params = rnn_model_source_to_target.parameters()
# mbart_params = group_parameters(rnn_model_target_to_source)
# if args.optimizer_back == 'adam':
rnn_optimizer_source_to_target = optim.AdamW(mbart_params)

num_update_steps_per_epoch = math.ceil(len(rnn_dataloader_train))
max_train_steps = num_update_steps_per_epoch*args.epochs
warmup_steps = 0.1*max_train_steps
rnn_scheduler_source_to_target = get_linear_schedule_with_warmup(
                rnn_optimizer_source_to_target, num_warmup_steps=warmup_steps, num_training_steps=max_train_steps
            )

 82%|████████▏ | 102/125 [00:06<00:01, 17.92it/s]


 65%|██████▍   | 81/125 [00:05<00:02, 18.05it/s]


 58%|█████▊    | 72/125 [00:04<00:03, 16.72it/s]


100%|██████████| 20/20 [02:31<00:00,  7.57s/it]


['Также Мерино под впечатлением от европейской литературы рисовал картины по произведениям Шекспира, Вальтера Скотта и Сервантеса.', 'В 2013 году объём производства окатышей превысил 10,4 млн тонн, а объем добычи породы — 50,5 млн м3.', 'На 1 января 2013 года численность населения составляла 5309 человек.', 'Когда они с матерью переехали в Крым в Коктебель (1893), Максимилиан пошёл в Феодосийскую гимназию (здание сохранилось, ныне в нём Феодосийская финансово-экономическая академия).', 'В ботанической литературе многолетнее растение обозначается символом Юпитера  ().']
['Тыйын колтымо да -влакым кычалашлан,ым терга.', 'нӧ а полдышым, да мутланымаш-влакымыште дене.', 'Пашаанеш-влакын ым кучылташ.', 'Тыйын але (web-влакым кычалашлан, садлан гына.', 'Эше да кн плагин-влак эре темдаым конден!']


TypeError: create_rnn_model() missing 2 required positional arguments: 'src_tok' and 'trg_tok'